<a href="https://colab.research.google.com/github/AnkitaSK/Proj3_RoboReviews/blob/main/ClusterProductCategoryOnReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import polars as pl

In [2]:
# Load large CSV file
df = pl.read_csv("1429_1.csv")

In [3]:
#print(df.head())  # Preview the first 5 rows
#print(df.shape)

In [4]:
reviews_text = df["reviews.text"]
reviews_text_list = [review for review in reviews_text if review is not None]
subset = reviews_text_list
#subset = reviews_text_list[2100:3200]
#print(subset)

In [5]:
#!pip install bertopic

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1,2), stop_words='english')

In [7]:
from sentence_transformers import SentenceTransformer

# custom embeddings
embedding_model = SentenceTransformer('all-MiniLM-l6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.1 MB/s eta 0:00:00


In [10]:
from umap import UMAP

umap_model = UMAP(
    n_neighbors=3,
    n_components=3,
    min_dist=0.05
    )

In [12]:
#!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 59.3 MB/s eta 0:00:00


In [13]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(
    min_cluster_size=80,
    min_samples=40,
    prediction_data=True,
    gen_min_span_tree=True
    )

In [15]:
#!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.7 MB/s eta 0:00:00


In [16]:
from bertopic import BERTopic

In [17]:
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    language='english',
    calculate_probabilities=True,
    verbose=True
    )
topics, probabilities = topic_model.fit_transform(subset)

2024-12-05 20:30:44,352 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1084 [00:00<?, ?it/s]

2024-12-05 20:30:56,445 - BERTopic - Embedding - Completed ✓
2024-12-05 20:30:56,446 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-05 20:31:25,297 - BERTopic - Dimensionality - Completed ✓
2024-12-05 20:31:25,299 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-05 20:31:33,801 - BERTopic - Cluster - Completed ✓
2024-12-05 20:31:33,814 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-05 20:31:35,818 - BERTopic - Representation - Completed ✓


In [18]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6890,-1_great_use_easy_love,"[great, use, easy, love, good, bought, product...",[I have been watching the reviews online and c...
1,0,5938,0_tablet_great_price_great tablet,"[tablet, great, price, great tablet, good, kid...",[Great tablet for the price. Good size. Easy t...
2,1,4023,1_kindle_read_reading_books,"[kindle, read, reading, books, paperwhite, lig...",[Best kindle ever! Easy to use and so beautifu...
3,2,3007,2_gift_product_bought_loves,"[gift, product, bought, loves, christmas, grea...",[Very great product very great product very gr...
4,3,2333,3_alexa_echo_music_amazon echo,"[alexa, echo, music, amazon echo, home, amazon...",[I use Alexa every day! From using the Amazon ...
5,4,1940,4_tv_stick_box_amazon tv,"[tv, stick, box, amazon tv, amazon, 4k, cable,...",[Best purchase I made this year. I got the Ama...
6,5,1542,5_read_reading_books_reader,"[read, reading, books, reader, light, easy, bo...",[Easy to use. Easy to read in any light. The b...
7,6,1461,6_music_sound_voice_speaker,"[music, sound, voice, speaker, great, weather,...",[Easy set up. Fun to use. Great sound quality....
8,7,807,7_echo_love echo_music_love,"[echo, love echo, music, love, weather, use, f...",[I love my Echo so much fun. the set up was so...
9,8,783,8_tablet_amazon_amazon tablet_hd,"[tablet, amazon, amazon tablet, hd, tablets, p...",[I love the Amazon Fire Tablet. It was easy to...


In [19]:
topic_model.get_topic(0)

[('tablet', 0.04631231405031425),
 ('great', 0.01660073115079075),
 ('price', 0.016447615629132016),
 ('great tablet', 0.014100789675310383),
 ('good', 0.013697814381818454),
 ('kids', 0.012083216142844235),
 ('use', 0.011529186870494933),
 ('apps', 0.011435221989790067),
 ('games', 0.010560650671027709),
 ('bought', 0.009600436144224579)]

In [22]:
topic_model.visualize_barchart(top_n_topics=10)

In [23]:
topic_model.visualize_topics()

In [24]:
topic_model.visualize_heatmap()

In [25]:
topic_model.visualize_hierarchy()